### Lab Machine Learning, del 2: Film rekommendations system
#### Daniel Claesson, ITHS, AI-23
[länk till dataset på Kaggle](https://grouplens.org/datasets/movielens/)
Det använda datasetet heter "ml-latest".<br>
Målet med uppgiften är att ge användaren rekommendationer på filmer som kan passa en användare.

In [1]:
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt

In [2]:
ratings = pd.read_csv('../data/ratings.csv')
movies = pd.read_csv('../data/movies.csv')
tags = pd.read_csv('../data/tags.csv')

#### EDA
- Läser in filerna ratings, movies och tags och gör en analys av vad filerna innehåller. Innehållet i filerna finns också beskrivet i readme filen som kommer med datasetet. EDA sammanfattas längre ned.

In [3]:
ratings.sample(10)
#ratings.info()
#ratings['userId'].nunique()
#ratings['movieId'].nunique()
#ratings.describe()

,userId,movieId,rating,timestamp
289079,2901,175661,4.5,1583077337
3961843,38668,765,2.0,864401048
33366422,326556,1732,1.5,1592619339
30440272,297626,1321,3.0,1507830368
13274916,130508,86320,3.0,1681323808
15697135,154361,590,2.5,1159293349
30628027,299397,1265,5.0,1025621083
19776403,193454,1356,1.5,1151957506
12975514,127337,164179,5.0,1539788334
7485019,73227,153,2.0,901043915


In [4]:
movies.sample(10)
#movies.info()

,movieId,title,genres
40042,157671,Manhattan Night (2016),Mystery|Thriller
76734,253916,The Green Sea (2021),Drama|Mystery
63555,207832,"Help, Help, the Globolinks! (1969)",Sci-Fi
22592,115614,Summertree (1971),Drama
30864,136786,Careful What You Wish For (2015),Thriller
57255,194070,Live (2018),Horror|Thriller
2697,2789,Damien: Omen II (1978),Horror
5994,6105,Moonlighting (1982),Drama
47204,172841,Longitude (2000),Drama
4256,4360,There's No Business Like Show Business (1954),Musical


In [5]:
movies[movies['movieId'] == 58559]

,movieId,title,genres
12223,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX


In [6]:
movies['genres'].value_counts().head(20)


genres
Drama                   12246
Documentary              8064
Comedy                   7689
(no genres listed)       7060
Comedy|Drama             3196
Drama|Romance            2788
Horror                   2467
Comedy|Romance           2176
Thriller                 1397
Comedy|Drama|Romance     1310
Drama|Thriller           1259
Horror|Thriller          1227
Crime|Drama              1142
Animation                1121
Drama|War                 811
Action                    783
Action|Drama              689
Western                   683
Crime|Drama|Thriller      651
Romance                   615
Name: count, dtype: int64

In [217]:
# vilka unika 'genres' finns det i movies? Delar upp genom att splitta på pipe.
movies['genres'].value_counts()

genres
Drama                                             12246
Documentary                                        8064
Comedy                                             7689
(no genres listed)                                 7060
Comedy|Drama                                       3196
                                                  ...  
Action|Adventure|Crime|Drama|Sci-Fi                   1
Adventure|Animation|Children|Comedy|Drama             1
Action|Adventure|Crime|IMAX                           1
Action|Adventure|Comedy|Crime|Mystery|Thriller        1
Action|Adventure|Drama|Horror|Mystery|Thriller        1
Name: count, Length: 1796, dtype: int64

In [216]:
#tags.sample(10)
#tags.info()
tags['movieId'].nunique()
#print(type(tags['tag'][1])) #str

53452

In [9]:
tags['tag'].value_counts().head(50)

tag
sci-fi                   14319
atmospheric              12172
action                   10683
comedy                   10161
surreal                   9142
funny                     9094
visually appealing        8890
twist ending              8325
thought-provoking         7727
dark comedy               7659
based on a book           7584
dystopia                  6975
cinematography            6473
social commentary         6369
romance                   6338
violence                  6300
stylized                  6163
psychology                6111
fantasy                   6082
murder                    5955
classic                   5839
quirky                    5822
dark                      5750
time travel               5716
adventure                 5427
coming of age             5194
suspense                  5121
friendship                4988
psychological             4970
thriller                  4949
drama                     4901
space                     4890
reve

In [10]:
# analys av hur många recensioner varje user har gjort:
#user_rating_count = ratings.groupby('userId')['rating'].count()
#user_rating_count_df = user_rating_count.reset_index(name='rating count')
#fig, ax = plt.subplots(figsize=(10,3.5))
#sns.histplot(user_rating_count_df, x='rating count')
#ax.set_xlim(0,300)


In [11]:
#ratings[ratings['userId'] == 126357]
ratings[ratings['userId'] == 62130]

,userId,movieId,rating,timestamp
6364967,62130,467,3.0,1663215326
6364968,62130,741,3.0,1662507230
6364969,62130,750,3.0,1655521157
6364970,62130,924,4.5,1662964251
6364971,62130,1197,3.5,1662504336
...,...,...,...,...
6365145,62130,258451,3.5,1655521502
6365146,62130,268642,2.0,1681204761
6365147,62130,270946,3.0,1681204750
6365148,62130,274053,2.0,1681204731


#### Analys "Dirty Dancing"

In [12]:
movies[movies['title'] == 'Dirty Dancing (1987)']

,movieId,title,genres
1061,1088,Dirty Dancing (1987),Drama|Musical|Romance


In [13]:
# analys av vilka användar taggar som en populär film som Dirt Dancing har fått:
tags_dd = tags[tags['movieId'] == 1088]
tags_dd

,userId,movieId,tag,timestamp
5649,1114,1088,dance,1147409630
5650,1114,1088,music,1147409627
17602,4311,1088,dancing,1367831310
22314,6407,1088,80's classic,1505669558
22315,6407,1088,coming of age,1505669556
...,...,...,...,...
2307520,327042,1088,Patrick Swayze,1166375572
2317033,328584,1088,80's classic,1391876140
2317034,328584,1088,dance,1391876127
2317035,328584,1088,dancing,1391876129


In [14]:
tags_dd['tag'].unique()

array(['dance', 'music', 'dancing', "80's classic", 'coming of age',
       'Nana viu', '1960s', 'Jennifer Grey', 'Patrick Swayze', 'romance',
       'soundtrack', 'summer camp', 'summer romance',
       'See:Dirty Dancing (1987)', 'AFI 100 (Movie Quotes)', 'overrated',
       'abortion', 'class differences', 'father daughter relationship',
       'teen movie', 'MMsWL', 'cheesy', 'chick flick', 'southern theme',
       'no one puts Baby in a corner', 'guilty pleasure', "80's",
       'musical parodies', 'classic', 'Dancing', 'bourgeois',
       'class themes', 'doctor', 'family holiday', 'holiday', 'holidays',
       'mambo', 'secret love', 'sister sister relationship',
       'unwanted pregnancy', 'Cynthia Rhodes', 'predictable',
       'social inequality', 'seen 2010', 'rich families', 'teen', 'cute',
       "1960's", '1963', 'girlie movie', 'BD-Video',
       'romance boygirl dancing', 'catskill resort', 'dance performance',
       'dancing master', 'daughter', 'hotel', 'robbery', '

In [15]:
tags_dd['tag'].value_counts().head(15)

tag
dancing           55
dance             32
coming of age     29
romance           26
Patrick Swayze    25
80's classic      15
music             15
cheesy            14
1960s             13
teen movie        11
abortion          10
Jennifer Grey      9
Dancing            9
classic            9
soundtrack         8
Name: count, dtype: int64

#### Analys "The Dark Knight"

In [16]:
movies[movies['title'] == 'Dark Knight, The (2008)']

,movieId,title,genres
12223,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX


In [17]:
tags_dk = tags[tags['movieId'] == 58559]
tags_dk

,userId,movieId,tag,timestamp
5,14,58559,Atmospheric,1311530439
6,14,58559,Batman,1311530391
7,14,58559,comic book,1311530398
8,14,58559,dark,1311530428
9,14,58559,Heath Ledger,1311530404
...,...,...,...,...
2328005,330824,58559,cinematography,1598820119
2328006,330824,58559,dark,1598819759
2328007,330824,58559,Hans Zimmer,1598820121
2328008,330824,58559,Heath Ledger,1598819766


In [18]:
tags_dk['tag'].unique()
#len(tags_dk['tag'].unique()) # 284

array(['Atmospheric', 'Batman', 'comic book', 'dark', 'Heath Ledger',
       'imdb top 250', 'Michael Caine', 'Morgan Freeman',
       'Oscar (Best Supporting Actor)', 'psychology', 'superhero',
       'vigilante', 'violence', 'action', 'Christopher Nolan',
       'Hans Zimmer', 'Heath Ledger as the Joker', 'Christian Bale',
       'atmospheric', 'cinematography', 'music',
       'Soundtrack: Hans Zimmer', 'great characters', 'great villain',
       'gritty', 'quotes', 'dark hero', 'mass surveillance', 'stylized',
       'Aaron Eckhart', 'based on a comic', 'Comic Book adaption',
       'DC Comics', 'Gary Oldman', 'Joker', 'Maggie Gyllenhaal', 'Quotes',
       'SuperVillain', 'thriller', 'too long', 'violent',
       'Library of Congress', 'National Film Registry', 'Batman (series)',
       'mask', 'To See', 'heath ledger', 'Dark hero', 'ensemble cast',
       'serial killer', 'anti-hero', 'Christopher Nolanï¼Œbatman',
       'corruption', 'organized crime', 'psychological thriller',
 

In [19]:
tags_dk['tag'].value_counts().head(20)

tag
Heath Ledger         257
Batman               243
dark                 212
superhero            202
Christian Bale       188
Christopher Nolan    178
action               160
psychology           149
Morgan Freeman       122
thriller             107
Atmospheric           79
Joker                 77
Michael Caine         77
atmospheric           74
vigilante             74
comic book            72
stylized              71
Gary Oldman           67
gritty                64
great villain         61
Name: count, dtype: int64

In [223]:
movies[movies['title'] == 'The Lord of the Rings (2001)']

,movieId,title,genres


#### Analys Batman Returns

In [20]:
movies[movies['title'] == 'Batman Returns (1992)']

,movieId,title,genres
1341,1377,Batman Returns (1992),Action|Crime


In [21]:
tags_br = tags[tags['movieId'] == 1377]
tags_br

,userId,movieId,tag,timestamp
6955,1398,1377,Gothic,1331174444
6956,1398,1377,superhero,1331174448
19330,4976,1377,super-hero,1151067311
19473,5143,1377,batman,1226116091
19474,5143,1377,superhero,1226116089
...,...,...,...,...
2189253,299033,1377,superhero,1411600821
2189254,299033,1377,Tim Burton,1411600819
2279745,318329,1377,super-hero,1137179137
2289267,321667,1377,franchise,1159463547


In [22]:
tags_br['tag'].value_counts().head(5)

tag
superhero            28
Tim Burton           27
Batman               22
dark hero            22
Michelle Pfeiffer    18
Name: count, dtype: int64

#### Analys "I'm not there"

In [23]:
movies[movies['title'] == "I'm Not There (2007)"]

,movieId,title,genres
11979,56286,I'm Not There (2007),Drama


In [24]:
tags_ImNotThere = tags[tags['movieId'] == 56286]
tags_ImNotThere

,userId,movieId,tag,timestamp
36391,7716,56286,androgyny,1226940972
99087,18266,56286,Bob Dylan,1675261319
136396,27227,56286,confusing,1307757786
136397,27227,56286,David Cross,1307757790
136398,27227,56286,surreal,1307757800
...,...,...,...,...
2086502,275825,56286,cate blanchett,1444512562
2222140,307012,56286,perrot library,1255144657
2282571,319758,56286,biography,1577661193
2282572,319758,56286,multiple storylines,1577661176


In [25]:
tags_ImNotThere['tag'].value_counts().head(20)

tag
Bob Dylan           12
surreal              7
surrealism           6
Mindfuck             6
confusing            6
music                5
androgyny            5
cate blanchett       5
Cate Blanchett       4
1960s                4
allen ginsberg       3
Christian Bale       2
bob dylan            2
horseback riding     2
faked death          2
actor                2
R                    2
David Cross          2
biography            2
reporter             2
Name: count, dtype: int64

### Mest populära tag's för alla filmer

In [26]:
#tags_list = tags['tag'].unique().tolist()
#len(tags_list) # 153950 unika tags

In [27]:
# skapar en lista över top_n tags baserat på hur ofta de förekommer bland alla filmer.
top_n = 10000 # 
top_n_tags = tags['tag'].value_counts().head(top_n).index.to_list()
print(f"Martin Scorsese finns bland top {top_n} användar-tags: {'Martin Scorsese' in top_n_tags}")
print(f"Bob Dylan finns bland top {top_n} användar-tags: {'Bob Dylan' in top_n_tags}")
print(f"Helena Bergström finns bland top {top_n} användar-tags: {'Helena Bergström' in top_n_tags}")

Martin Scorsese finns bland top 10000 användar-tags: True
Bob Dylan finns bland top 10000 användar-tags: True
Helena Bergström finns bland top 10000 användar-tags: False


#### EDA slutsatser, summering och tankar
- 'ratings' innehåller 33.8 miljoner filmbetyg ('rating') från 0 till 5 med steget 0.5, av 330.975 unika användare ('userId'), för 83.239 unika filmer ('movieId').
- 'movies' innehåller data för 86.532 unika filmer ('movieId'). Innehåller titel ('title') och genres ('genres'). Genres är en pipe-separated lista som genres som passar till filmen. 7060 filmer saknar helt 'genre'.
- 'tags' innehåller 2.328.315 användar-adderade taggar ('tag') till filmerna, användarna identifieras med ('userId'). 53.452 av filmerna har fått taggar av användarna. Dvs, cirka 30.000 filmer saknar användar-adderade taggar. Det finns 153.950 unika tags.
- Alla ratings och tags har också en timestamp. Har inte undersökt denna mer noggrant än så.
- Det finns en spridning i hur många ratings användare har gjort. Många som gjort en rating, har endast gjort en (1 rating). En del användare har gort många ratings. Det är en topp kring runt 20-25 ratings.
<br>
- I filen 'ratings' finns kolumnen 'tags', som användare har angett. Den informationen kommer vara essentiell, eftersom baserat på den informationen anger mycket information om filmen. Eftersom man vet vilka användare som skrivit tags, så kan eventuellt informationen kan avslöja användarens preferenser, men det behöver troligen undersökas.
- Att använda 'tags' kan också vissa innebära svårigheter. Tex filmen Dark Knight har 'batman' 243 gånger men Batman Returns har 'batman' endast 22 gånger. Dvs, datan måste normaliserar eller liknande (Dark Knight är ju inte 11 gånger mer 'batman' än Batman Returns). Samtidigt måste troligen hänsyn tas till 'frekvens' av olika tags: Dark Knight är ju mer 'batman' än 'great villain'.
- En annan observation är att informationen i 'genre' kolumnen i filen 'movies' är tämligen begränsad. Det styrker att användar-taggarna blir avgörande för att kunna ge bra rekommendationer. Detta eftersom de ger en djupare insikt i vad varje film egentligen handlar om, vilka skådespelare som spelar osv. Dessutom, många taggar är återkommande, vilket gör att man kan rangordna taggar utefter hur ofta de förekommer.
    - Ett exempel på att användartaggarna kommer vara avgörande är filmen The Dark Knight. I 'movies' datan så anges endast "Action|Crime|Drama|IMAX" i 'genre'. Det är väldigt begränsad information och säger endast på en väldigt hög nivå vad filmen handlar om (tex kan man utesluta kommedi eller western). När man adderar användar taggarna så får man information såsom 'batman', 'Heath Ledger', 'super hero, 'joker' med mera. Denna informationen är också omfattande, det finns 284 unika tags som tillsammans ger mycket mer kött på benen vad filmen handlar om.
    - Man skulle kunna sammanfatta det med att taggarna ger en stor mängd information, liknande Features i analys vi gjort tidigare i ML kursen. Men Features'arna kommer vara unika för varje film, men givetvis kan samma tagg återkomma i olika filmer. Alla Batman filmer har ju troligen taggen 'batman'.
    - Om man bestämmer sig för att klustra filmer baserat på "hur lika de är varandra" kommer taggarna vara nödvändiga.


- Mina egna tankar kring hur man skulle kunna göra:
1. En strategi skulle kunna vara att rekommendera filmer till en användare baserat på den användarens och andra användares ratings av filmer. Dvs, klustra användare, så att liknande användare hamnar i samma kluster. Det kräver förmodligen att användaren har gjort ganska många ratings, eftersom det krävs för att definiera användaren och placera i rätt kluster (detta kallas "cold start problem" enligt Wikipedia artikeln). Samma gäller för filmer (dvs, filmer med få ratings kommer vara svåra att rekommendera). Användare som gjort liknande ratings hamnar i samma kluster. I dess enklaste form, skulle man sedan rekommendera filmer till användaren, som andra användare i samma kluster har gett högt betyg, men som användaren själv inte betygsatt (="inte sett själv").
2. En annan strategi skulle kunna vara att rekommendera filmer till en användare baserat på filmer som användaren gillar. Användaren skulle då ange titel på sin favoritfilm eller favoritfilmer, för att sedan få förslag på liknande filmer. Algoritmen skulle kunna ta hänsyn till vilka betyg liknande filmer har fått och prioritera filmer med högt betyg som förslag till användaren. I den här strategin skulle man klustra filmer baserat på likhet, genom att ta hänsyn till filmens 'genre' och användartaggar för filmen.
3. En tredje strategi som kombinerar användarens betyg för olika filmer med filmernas 'features' skulle kunna vara att identifiera användarens smak mha filmernas tags. Tex kanske en user ger höga betyg till filmer som taggats 'Sci-Fi' men låga till filmer som taggats 'western'.

- Existerande metoder:
- Collaborative filtering: Denna metoden liknar min tanke 1 ovan. Metoden går ut på att man kan uppskatta en användare A's rating av osedda filmer, baserat på andra liknande användares rating på dessa filmer. I princip kommer man använda rating av andra användare med liknande smak, för att uppskatta en rating för filmer som användare A inte har betygsatt.<br>Wikipedia: "Den bästa rekommendationen kommer från andra med liknande smak."  
[Wikipedia](https://en.wikipedia.org/wiki/Collaborative_filtering)  
[knowledgehut](https://www.knowledgehut.com/blog/data-science/movie-recommendation-system)  
[algolia](https://www.algolia.com/blog/ai/the-anatomy-of-high-performance-recommender-systems-part-iv/?utm_source=google&utm_medium=paid_search&utm_campaign=rl_emea_search_dstalg_nb_dynamic&utm_content=blog_ai_dynamic&utm_term=&utm_region=emea&utm_model=nonbrand&utm_ag=rl&utm_camp_parent=slg&utm_2nd_camp=dstalg&_bt=677640514505&_bm=&_bn=g&gad_source=1&gclid=CjwKCAjwzN-vBhAkEiwAYiO7oMYf76up9PTTg7tuoqBRKsJyAohMi_ZiP-Yow0JtYB0T7T1e8meiEhoCfVMQAvD_BwE)  
<br>
- Content filtering: Metoden går ut på att hitta likheter mellan filmer. Den kunskapen kan sedan användas för att, baserat på en film-titel hitta liknande filmer. Så om användaren anger en (eller flera) film-titel som den gillar, kan algoritmen ge förslag på andra liknande filmer. Eftersom taggarna är essentiella för uppdraget, så behöver vi ha en metod som kan jämföra taggar för olika filmer och kunna avgöra om taggarna liknar varandra eller om de skiljer sig avsevärt. En sådan teknik är "cosine similarity".


- Rekomenderingsalgoritmen bör (troligen) helst i första hand rekommendera 'populära' filmer. Dvs filmer som har många recensioner och höga betyg (men som inte blivit recenserad av användaren). Men för optimal prestanda måste den också kunna ge förslag på mindre kända filmer, som kanske inte har många recensioner men ett stadigt högt betyg eller som uppfyller användarens nischade smak.

    

#### Approach
- Jag försöker göra en Collaborative Filtering approach.
- 

#### Skapar en sparse matrix
- väljer ut m top filmer och n random users, för att minska ner mängden information, så den blir överskådlig.

In [28]:
# slumpar fram n userId's (alla har troligen inte gett ratings på filmer)
import random
random.seed(42)
n = 50
list_of_all_users = ratings['userId'].unique().tolist()
random_n_users =random.sample(list_of_all_users, n)
print(random_n_users)

[58370, 13113, 144195, 128394, 117027, 73159, 53737, 285930, 45581, 309589, 221209, 16664, 15623, 49124, 114630, 121982, 264952, 315632, 13913, 294255, 104249, 285707, 219950, 115575, 235515, 308945, 145853, 3408, 83708, 221572, 178390, 145685, 81517, 112887, 176473, 53588, 48626, 199192, 50708, 188209, 180332, 316527, 138686, 22781, 240871, 281138, 65445, 198462, 41314, 289429]


In [29]:
reduced_ratings = ratings[ratings['userId'].isin(random_n_users)]
reduced_ratings['userId'].unique() # de här userID'na har gett rating på minst en film

array([  3408,  13113,  13913,  15623,  16664,  22781,  41314,  45581,
        48626,  49124,  50708,  53588,  53737,  58370,  65445,  73159,
        81517,  83708, 104249, 112887, 114630, 115575, 117027, 121982,
       128394, 138686, 144195, 145685, 145853, 176473, 178390, 180332,
       188209, 198462, 199192, 219950, 221209, 221572, 235515, 240871,
       264952, 281138, 285707, 285930, 289429, 294255, 308945, 309589,
       315632, 316527], dtype=int64)

In [30]:
reduced_ratings.sample(10)

,userId,movieId,rating,timestamp
341396,3408,3034,4.0,1062887913
18407312,180332,1213,4.5,1107304278
32294393,315632,786,3.0,856087008
28779525,281138,736,3.0,846871860
19205944,188209,2105,1.0,901504810
29242952,285707,116823,3.5,1443513379
11748094,115575,1748,1.5,1135704252
24725557,240871,1302,5.0,907048233
8529393,83708,104,2.0,863277614
5237639,50708,6377,4.0,1159377617


In [31]:
# väljer ut de m mest betygsatta filmerna i datasetet
m = 25
top_m_movies = ratings['movieId'].value_counts().head(m).index.to_list()
#top_m_movies # filmens "movieId"

In [32]:
# reducerar reduced_ratings så att den enbart innehåller de m mest rateade filmerna
reduced_ratings_2 = reduced_ratings[reduced_ratings['movieId'].isin(top_m_movies)]

In [33]:
# skapa en sparse matrix, baserat på de n users och m filmer med Pandas pivot method och scipys csr_matrix:
from scipy.sparse import csr_matrix
pivot_table = pd.pivot_table(data=reduced_ratings_2, index='userId', columns='movieId', values='rating', fill_value=0)
pivot_table

movieId,1,47,50,110,260,296,318,356,480,527,...,1210,1270,2571,2858,2959,4993,5952,7153,58559,79132
userId,,,,,,,,,,,,,,,,,,,,,
3408,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13913,0.0,5.0,4.0,0.0,0.0,5.0,5.0,5.0,4.5,5.0,...,3.0,5.0,5.0,5.0,4.5,0.0,0.0,0.0,0.0,0.0
16664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22781,0.0,2.0,0.0,5.0,0.0,2.0,0.0,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41314,0.0,0.0,0.0,0.0,0.0,0.0,4.5,4.5,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45581,0.0,0.0,3.5,0.0,3.5,4.5,4.5,0.0,0.0,3.5,...,0.0,0.0,4.5,4.0,4.5,3.5,3.5,3.5,0.0,0.0
49124,0.0,5.0,5.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,0.0,0.0,5.0,4.0,5.0,0.0,0.0,3.0,5.0,5.0
53737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
sparse_matrix = csr_matrix(pivot_table)
#print(sparse_matrix)

In [35]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=20)
user_features = svd.fit_transform(sparse_matrix)
user_features

array([[ 4.22415432e+00, -2.01589844e+00, -7.33922298e-01,
         2.82707530e+00, -1.27532040e+00, -1.08037786e+00,
         1.68380283e+00,  2.93765171e+00,  1.35771644e+00,
         1.97506772e+00,  2.15972992e+00, -2.24701400e+00,
         1.06973095e+00,  8.52245620e-01,  7.30583106e-01,
         2.34797197e-01,  7.77755055e-01,  5.15860787e-02,
         4.54082719e-01,  4.61166083e-01],
       [ 6.64453695e-01,  9.49262407e-02,  1.82847309e+00,
        -4.31379757e-01,  1.72163595e-01,  1.38199347e+00,
         2.56864215e+00, -1.57998180e-01, -8.96020867e-01,
        -9.80357440e-01, -1.16379702e+00, -1.34863385e-01,
         2.94748767e+00,  4.32049252e-01, -2.31165538e-01,
        -9.30768576e-02,  3.69266057e-02,  1.70058465e-01,
        -1.68157764e-01, -5.36560821e-01],
       [ 1.69477377e+01,  2.34960196e+00, -2.04755451e+00,
         5.70882085e+00, -3.71654603e-01,  5.01251420e+00,
         1.46026185e+00,  7.90754053e-02,  1.31326366e+00,
        -6.55685821e-01,  1.0

Min tanke har varit att jag ska kunna uppskatta de saknade värdena i sparse matrisen, baserat på hur övriga userId'n har betygsatt. Detta är ett försök men det är oklart om detta verkligen kan ge något bra resultat.
Får se vad nästa steg blir.

### Content Filtering försök:
- Jag gör en annan approach, enligt min beskrivning nummer 2 ovan.
- Tanken är att klustra filmer baserat på hur lika de är varandra. Algoritmen som jag provar är K-means. 'tag' och 'genre' används för att särskilja filmerna.
- En dataframe / matris med alla tags / genres som index och alla filmer som kolumner, måste skapas som första steg. Detta behöver vara en snabb funktion eftersom det är så många filmer och tags att gå igenom.
- Jag börjar med ett 'm' av filmer och ett 'n' antal parametrar, bara för att se om det ens fungerar.

In [203]:
m = 85000
top_m_movies = ratings['movieId'].value_counts().head(m).index.to_list()

In [207]:
n= 10000
tags_top_n = tags['tag'].value_counts().head(n)
tags_top_n_list = tags['tag'].value_counts().head(n).index.to_list()
tags_top_n

tag
sci-fi                            14319
atmospheric                       12172
action                            10683
comedy                            10161
surreal                            9142
                                  ...  
social services                      26
talking to oneself in a mirror       26
reference to napoleon                26
american south                       26
rudeness                             26
Name: count, Length: 10000, dtype: int64

Reducerar dataframen med tags till att enbart innehålla kombinationen av de 'm' mest populära filmerna samt de 'n' mest använda taggarna.

Med hjälp av pivot_table funktionen skapas den ovan beskrivna tabellen, utifrån den reducerade tags dataframen.
- som rader är alla 'movieId'
- som kolumner är alla 'tag'
- funktionen 'size' summerar antalet gånger en tag förekommer. Tex för The Dark Knight så förekommer 'Heath Ledger' som tag 257 gånger, vilket är samma i nedan matris.

In [208]:
tags_reduced = tags[(tags['tag'].isin(tags_top_n_list)) & (tags['movieId'].isin(top_m_movies))]

In [209]:
pivot_table = pd.pivot_table(data=tags_reduced, index='movieId', columns='tag', aggfunc='size', fill_value=0)
pivot_table

tag,"""found footage""",(s)vcd,*,007,007 (series),01/10,01/11,01/12,02/10,02/11,...,yuppie,z movie,zeppelin,zero gravity,zippo lighter,zombie,zombie apocalypse,zombie attack,zombies,zoo
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288765,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Nedan kan man se att 'Heath Ledger' har värdet 257 i pivot_table dataframen (vilket är betryggande, vi är på rätt väg.)

In [210]:
pivot_table.loc[58559,:].sort_values() # kollar The Dark Knight

tag
"found footage"             0
miscarriage                 0
miscarriage of justice      0
misery                      0
misfit                      0
                         ... 
Christian Bale            188
superhero                 202
dark                      212
Batman                    243
Heath Ledger              257
Name: 58559, Length: 10000, dtype: int64

Nästa steg är att klustra liknande filmer med K-means.
- gör scaling först.

In [211]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#scaler = StandardScaler()
scaler = MinMaxScaler()
scaled_pivot_table = scaler.fit_transform(pivot_table)
scaled_pivot_table

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Nedan provar jag lite olika 'k', dvs antal kluster. Antalet kluster man vill använda kan nog variera lite, men man kan säkert göra 100 olika kluster. Men efter runt 250 kluster så sjunker inte inertia så mycket.

In [212]:
from sklearn.cluster import KMeans

#clusters = range(100,500,50)
#sum_squared_distance = [
#    KMeans(k, n_init="auto").fit(scaled_pivot_table).inertia_ for k in clusters
#]
#sum_squared_distance

#KMeans(200, n_init="auto").fit(scaled_pivot_table).inertia_

In [213]:
k = 250
kmeans = KMeans(k, n_init="auto").fit(scaled_pivot_table)

In [214]:
labels = pd.DataFrame(kmeans.labels_)
labels.value_counts().head(19)

0  
7      41214
78      3068
81      1018
131      379
96       285
59       245
65       221
2        220
150      162
152       93
25        92
6         82
183       80
80        79
127       71
236       66
0         59
35        58
53        56
Name: count, dtype: int64

Jag har ett problem här. Det är så att storleken på klustren är väldigt olika. Det största klustret innehåller 6000-7000 filmer (klustrets storlek varierar beroende på värdet på k, jag har provat med 100 - 250 kluster ungefär). Sedan är det relativt få kluster med fler än sådär 20-30 filmer upp till 1000 filmer. De flesta kluster innehåller 1-2 filmer. Här skulle man vilja ha jämnare storlek på klustren. Ett försök till det, kan vara att även utnyttja informationen om 'genre' (vilken vi inte utnyttjat ännu).
Innan jag börjar jobba med att addera 'genre', så kollar jag att skalningen blev bra. Om den inte blev det kan jag tänka mig att det kan bli tokigt.

Skapar en dataframe igen så att jag kan kolla hur sakerna ser ut efter skalningen. Eftersom jag får i princip några få stora kluster vill jag bara kolla att skalningen är rätt.

In [215]:
scaled_pivot_table_df = pd.DataFrame(data=scaled_pivot_table, columns=pivot_table.columns, index=pivot_table.index )
scaled_pivot_table_df

tag,"""found footage""",(s)vcd,*,007,007 (series),01/10,01/11,01/12,02/10,02/11,...,yuppie,z movie,zeppelin,zero gravity,zippo lighter,zombie,zombie apocalypse,zombie attack,zombies,zoo
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Adderar 'genre' till analysen
- många filmer, tex The Dark Knight har flera genres separerade med en pipe, typ  (Drama|Commedy|Action). Jag tror att det bästa är att göra en ny dataframe där varje genre får en "egen" rad, för att öka chansen att liknande filmer får mer lika features.

In [193]:
movies_reduced = movies[movies['movieId'].isin(top_m_movies)] # tar bara med de top m filmer vi analyserar, valt ovan
movies_reduced['genres'] = movies_reduced['genres'].str.split('|')
movies_exploded = movies_reduced.explode('genres')

C:\Users\danie\AppData\Local\Temp\ipykernel_11816\2117182021.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reduced['genres'] = movies_reduced['genres'].str.split('|')


In [194]:
movies_exploded.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy
1,2,Jumanji (1995),Adventure
1,2,Jumanji (1995),Children
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy
2,3,Grumpier Old Men (1995),Romance


Adderar alla unika 'genres' till en lista så att jag kan använda den nedan för att ta bort dubbletter (dvs, ta bort "action" från användar tags, om "action" även finns bland 'genres')

In [224]:
genres_list = movies_exploded['genres'].unique().tolist()
genres_list

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'IMAX',
 'Documentary',
 'War',
 'Musical',
 'Western',
 'Film-Noir',
 '(no genres listed)']

Skapar en pivot table, liknande den för 'tags', men för 'genre'.

In [196]:
pivot_table_genre = pd.pivot_table(data=movies, index='movieId', columns='genres', aggfunc='size', fill_value=0)
pivot_table_genre

genres,(no genres listed),Action,Action|Adventure,Action|Adventure|Animation,Action|Adventure|Animation|Children,Action|Adventure|Animation|Children|Comedy,Action|Adventure|Animation|Children|Comedy|Fantasy,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure|Animation|Children|Comedy|Romance,Action|Adventure|Animation|Children|Comedy|Sci-Fi,...,Sci-Fi|Thriller|War,Sci-Fi|Thriller|Western,Sci-Fi|War,Sci-Fi|Western,Thriller,Thriller|War,Thriller|Western,War,War|Western,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Denna dataframen är inte samma storlek som den förra (den innehåller fler filmer), så jag måste mergea dem på gemensamt movieId (och ev ta bort NaN's).

In [197]:
scaled_pivot_table_df.merge(pivot_table_genre, left_on='movieId', right_on='movieId')

,action,atmospheric,comedy,dark comedy,funny,sci-fi,surreal,thought-provoking,twist ending,visually appealing,...,Sci-Fi|Thriller|War,Sci-Fi|Thriller|Western,Sci-Fi|War,Sci-Fi|Western,Thriller,Thriller|War,Thriller|Western,War,War|Western,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.003279,0.000000,0.24375,0.0,0.508621,0.000911,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.000000,0.03125,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.000000,0.00625,0.0,0.008621,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
6,0.003279,0.261307,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,0.000000,0.000000,0.00625,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287633,0.000000,0.000000,0.00000,0.0,0.000000,0.000911,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
288167,0.003279,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
288265,0.003279,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


KOllar om det finns NaN's, och det finns inga. Bra.

In [198]:
True in scaled_pivot_table_df.isna()

False

Nu är 'genre' informationen adderad. Kör K-Means igen.

In [199]:
k = 250
kmeans_final = KMeans(k, n_init="auto").fit(scaled_pivot_table_df)

In [200]:
labels = pd.DataFrame(kmeans_final.labels_)
labels.value_counts().head(19)

0  
88     1374
81      627
0       481
138     295
147     244
174     226
62      117
155     104
175      96
95       85
216      75
242      73
33       71
171      65
185      57
80       56
142      55
241      46
190      45
Name: count, dtype: int64